# 檢測GPU

In [1]:
import torch
print('GPU:', torch.cuda.is_available())

GPU: True


# 亂數模擬資料

In [2]:
import pandas as pd

# 讀取 CSV 檔案並印出前幾筆資料
df = pd.read_csv('simulated_data.csv')

loaded_data = df.iloc[:, :-1].values  # 所有特徵欄位
loaded_labels = df.iloc[:, -1].values  # 標籤欄位

for i in range(2):
    print('-' * 60)
    print(f'Loaded sample [{i}]')
    print(f'  Features : {loaded_data[i].tolist()}')
    print(f'  Label    : {loaded_labels[i]}')
print('-' * 60)
print(f'Loaded data shape : {loaded_data.shape}')  # 使用shape一定要是array或tensor
print(f'Loaded labels shape: {loaded_labels.shape}')

------------------------------------------------------------
Loaded sample [0]
  Features : [10, 37, 48, 45, 12, 44, 12]
  Label    : 1
------------------------------------------------------------
Loaded sample [1]
  Features : [19, 17, 6, 12, 39, 11, 21]
  Label    : 0
------------------------------------------------------------
Loaded data shape : (10000, 7)
Loaded labels shape: (10000,)


# 正規化資料

In [3]:
# 找到資料集的上下限
data_min = loaded_data.min()
data_max = loaded_data.max()
print('-' * 60)
print(f'Data Range')
print(f'  Min    : {data_min}')
print(f'  Max    : {data_max}')
print('-' * 60)

------------------------------------------------------------
Data Range
  Min    : 1
  Max    : 49
------------------------------------------------------------


In [4]:
def normalize_sequences(sequences):
    min_val = sequences.min()
    max_val = sequences.max()
    return (sequences - min_val) / (max_val - min_val)

# 正規化處理
normalized_data = normalize_sequences(loaded_data)

# 顯示前後
sample_index = 0
print('-' * 60)
print(f'Original sample [{sample_index}]')
print(f'  Data   : {[f"{v:.5f}" for v in loaded_data[sample_index].tolist()]}')
print('-' * 60)
print(f'Normalized sample [{sample_index}]')
print(f'  Data   : {[f"{v:.5f}" for v in normalized_data[sample_index].tolist()]}')
print('-' * 60)

------------------------------------------------------------
Original sample [0]
  Data   : ['10.00000', '37.00000', '48.00000', '45.00000', '12.00000', '44.00000', '12.00000']
------------------------------------------------------------
Normalized sample [0]
  Data   : ['0.18750', '0.75000', '0.97917', '0.91667', '0.22917', '0.89583', '0.22917']
------------------------------------------------------------


In [5]:
from sklearn.model_selection import train_test_split

# 使用 sklearn 分割資料
train_data, val_data, train_labels, val_labels = train_test_split(
    normalized_data, loaded_labels, test_size=0.2, random_state=42
)

# 顯示資料切割
print('-' * 60)
print('Train data count      :', len(train_data))
print('Validation data count :', len(val_data))
print('-' * 60)

------------------------------------------------------------
Train data count      : 8000
Validation data count : 2000
------------------------------------------------------------


In [6]:
import torch
from torch.utils.data import Dataset

# 自訂資料集類別，回傳可變長度序列
class MyDataset(Dataset):
    def __init__(self, data, labels):
        # 初始化資料與標籤
        self.data = data
        self.labels = labels

    def __len__(self):
        # 回傳資料集的大小
        return len(self.data)

    def __getitem__(self, idx):
        # 回傳 dict 格式包含序列與標籤
        return {
            'input': self.data[idx],
            'target': self.labels[idx]
        }

# 建立資料集實例
train_dataset = MyDataset(train_data, train_labels)
val_dataset = MyDataset(val_data, val_labels)

# 顯示資料集中第一筆樣本資訊
sample = train_dataset[0]
print('\n' + '-' * 60)
print('Sample from train_dataset')
print(f"{'Input':<8}- Shape : {len(sample['input'])}\n{'':<10}Data  : {sample['input']}")
print(f"{'Target':<8}- Label : {sample['target']}")
print('-' * 60 + '\n')



------------------------------------------------------------
Sample from train_dataset
Input   - Shape : 7
          Data  : [0.04166667 0.95833333 0.83333333 0.10416667 0.52083333 0.45833333
 0.60416667]
Target  - Label : 1
------------------------------------------------------------



In [7]:
def my_collate_fn(batch):
    inputs  = [item['input'] for item in batch]
    targets = [item['target'] for item in batch]
    
    return {
        'input':  torch.tensor(inputs, dtype=torch.float32),
        'target': torch.tensor(targets, dtype=torch.long)
    }

# 使用 sample 套用自訂的 collate_fn，這裡取第 0 筆與第 1 筆資料
batch           = [train_dataset[0], train_dataset[1]]
collated_sample = my_collate_fn(batch)

print('\n' + '-' * 60)
print('Sample after applying my_collate_fn\n')
print(f"{'Input':<10}- Shape : {collated_sample['input'].shape}")
print(f"{'':<10}  Data  :\n{collated_sample['input']}\n")
print(f"{'Target':<10}- Label : {collated_sample['target']}")
print('-' * 60 + '\n')



------------------------------------------------------------
Sample after applying my_collate_fn

Input     - Shape : torch.Size([2, 7])
            Data  :
tensor([[0.0417, 0.9583, 0.8333, 0.1042, 0.5208, 0.4583, 0.6042],
        [0.5417, 0.3958, 0.8125, 0.2083, 0.1250, 0.9792, 0.7500]])

Target    - Label : tensor([1, 1])
------------------------------------------------------------



C:\Users\user\AppData\Local\Temp\ipykernel_36692\2900401265.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  'input':  torch.tensor(inputs, dtype=torch.float32),


In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset    = train_dataset,
    batch_size = 5,
    shuffle    = True,
    collate_fn = my_collate_fn
)

valid_loader = DataLoader(
    dataset    = val_dataset,
    batch_size = 5,
    shuffle    = False,
    collate_fn = my_collate_fn
)

# 取出一個 batch 的資料進行檢查（只取第一個）
collated_sample = next(iter(train_loader))
print('\n' + '-' * 60)
print('Sample after applying my_collate_fn with DataLoader\n')
print(f"{'Input':<10}- Shape : {collated_sample['input'].shape}")
print(f"{'':<10}  Data  :\n{collated_sample['input']}\n")
print(f"{'Target':<10}- Label : {collated_sample['target']}")
print('-' * 60 + '\n')



------------------------------------------------------------
Sample after applying my_collate_fn with DataLoader

Input     - Shape : torch.Size([5, 7])
            Data  :
tensor([[0.1250, 0.1667, 0.0625, 0.4583, 0.1250, 0.7292, 0.7708],
        [0.2708, 0.5417, 0.8542, 0.1458, 0.8958, 0.7917, 0.4792],
        [0.4167, 0.8750, 0.0000, 0.2292, 0.1250, 0.2500, 0.2292],
        [0.0417, 0.3750, 0.8542, 0.3333, 0.0000, 0.3750, 0.6250],
        [0.5417, 0.6875, 0.3958, 0.0625, 0.5625, 0.8333, 0.2708]])

Target    - Label : tensor([0, 1, 0, 0, 0])
------------------------------------------------------------



In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 建立 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden)  # 全連接層，將輸入從 input_size 維轉換為 hidden 維
        self.fc2 = nn.Linear(hidden, output_size)  # 全連接層，輸出為 output 類別
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, **kwargs):
        x = self.fc1(kwargs['input'])  # 第一次線性變換
        x = F.relu(x)  # ReLU 非線性轉換
        x = self.fc2(x)  # 第二次線性變換，輸出 logits
        loss = self.loss_fn(x, kwargs['target'])  # 計算 loss
        
        return loss, x

model = MLP(input_size=train_data.shape[1], hidden=4, output_size=2)

In [10]:
# 訓練函數
def train_one_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in dataloader:
        optimizer.zero_grad()
        loss, outputs = model(**batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct += (predicted == batch['target']).sum().item()
        total += batch['target'].size(0)
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

# 驗證函數
def validate(model, dataloader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            loss, outputs = model(**batch)
            total_loss += loss.item()
            predicted = outputs.argmax(dim=1)
            correct += (predicted == batch['target']).sum().item()
            total += batch['target'].size(0)
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

# 執行訓練流程
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, valid_loader)
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'.format(
        epoch + 1, num_epochs, train_loss, train_acc, val_loss, val_acc))

Epoch [1/10], Train Loss: 0.6316, Train Acc: 0.6465, Val Loss: 0.5349, Val Acc: 0.7870
Epoch [2/10], Train Loss: 0.4126, Train Acc: 0.9217, Val Loss: 0.3371, Val Acc: 0.9140
Epoch [3/10], Train Loss: 0.2701, Train Acc: 0.9666, Val Loss: 0.2358, Val Acc: 0.9890
Epoch [4/10], Train Loss: 0.2014, Train Acc: 0.9774, Val Loss: 0.1873, Val Acc: 0.9750
Epoch [5/10], Train Loss: 0.1628, Train Acc: 0.9814, Val Loss: 0.1577, Val Acc: 0.9710
Epoch [6/10], Train Loss: 0.1388, Train Acc: 0.9829, Val Loss: 0.1362, Val Acc: 0.9820
Epoch [7/10], Train Loss: 0.1219, Train Acc: 0.9879, Val Loss: 0.1201, Val Acc: 0.9905
Epoch [8/10], Train Loss: 0.1094, Train Acc: 0.9870, Val Loss: 0.1093, Val Acc: 0.9910
Epoch [9/10], Train Loss: 0.1001, Train Acc: 0.9880, Val Loss: 0.1006, Val Acc: 0.9915
Epoch [10/10], Train Loss: 0.0921, Train Acc: 0.9885, Val Loss: 0.0982, Val Acc: 0.9740


In [11]:
import torch
import torch.nn as nn


def run_backward_and_print(title, clear_grad):
    if clear_grad:
        optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    print(title, model.weight.grad)

# 建立一個簡單模型
model = nn.Linear(2, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# 假資料
x = torch.tensor([[1.0, 2.0]], requires_grad=True)
y = torch.tensor([[1.0]])

run_backward_and_print("第一次梯度：", clear_grad=True)
run_backward_and_print("第二次未清除梯度：", clear_grad=False)
run_backward_and_print("第三次清除後的梯度：", clear_grad=True)


第一次梯度： tensor([[1.4492, 2.8984]])
第二次未清除梯度： tensor([[2.8984, 5.7968]])
第三次清除後的梯度： tensor([[1.4492, 2.8984]])
